In [49]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from bs4 import BeautifulSoup
import requests
import pandas as pd


In [50]:
df = pd.read_csv('Dataset1.csv')

In [51]:
def replace_wrong_colors (wrong_color, proper_color): # Функция испралвения имён
  for wrong_color in wrong_color:
    df['color'] = df['color'].replace(wrong_color, proper_color)

df=df.rename(
    columns = {
        'Director_Name' : 'director_name',
        'num_Critic_for_reviews' : 'num_critic_for_reviews',
        'director_Facebook_likes' : 'director_facebook_likes',
        'actor_3_Facebook_likes' : 'actor_3_facebook_likes',
        'Actor_1_Facebook_likes' : 'actor_1_facebook_likes',
        'movie_Title' : 'movie_title',
        ' cast_total_facebook_likes' : 'cast_total_facebook_likes',
        'facenumber_in_poster' : 'face_number_in_poster',
        'movie_facebook_likes;' : 'movie_facebook_likes'
    }
)

df = df.drop_duplicates().reset_index(drop=True) # Перезапись индексов
duplicates = ['colour', 'Colour', 'Color']
name = 'color'
replace_wrong_colors(duplicates, name) # Удаление неявных дубликатов

df.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000;
1,color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0;
2,color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000;
3,color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000;
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0;


In [53]:
value_columns = [
    'aspect_ratio',
    'imdb_score',
    'actor_2_facebook_likes',
    'title_year',
    'budget',
    'num_user_for_reviews',
    'face_number_in_poster',
    'cast_total_facebook_likes',
    'num_voted_users',
    'gross',
    'actor_1_facebook_likes',
    'actor_3_facebook_likes',
    'director_facebook_likes',
    'duration',
    'num_critic_for_reviews'
]

string_columns = [
    'content_rating',
    'country',
    'language',
    'movie_imdb_link',
    'actor_3_name',
    'movie_title',
    'actor_1_name',
    'actor_2_name',
    'director_name',
    'color',
    'movie_facebook_likes',
    'plot_keywords',
    'genres'
]

for row in string_columns:
    df[row] = df[row].fillna('unknown')

for row in value_columns:
    df[row] = df[row].fillna(df[row].median()) # Медианное значение (посередние)

df.isna().sum()

color                        0
director_name                0
num_critic_for_reviews       0
duration                     0
director_facebook_likes      0
actor_3_facebook_likes       0
actor_2_name                 0
actor_1_facebook_likes       0
gross                        0
genres                       0
actor_1_name                 0
movie_title                  0
num_voted_users              0
cast_total_facebook_likes    0
actor_3_name                 0
face_number_in_poster        0
plot_keywords                0
movie_imdb_link              0
num_user_for_reviews         0
language                     0
country                      0
content_rating               0
budget                       0
title_year                   0
actor_2_facebook_likes       0
imdb_score                   0
aspect_ratio                 0
movie_facebook_likes         0
dtype: int64

## **Преобразуем валовую и бюджетную стоимость в реальные суммы в долларах с точки зрения покупательной способности 2016 года.**

---

In [54]:
url = "https://www.usinflationcalculator.com/inflation/consumer-price-index-and-annual-percent-changes-from-1913-to-2008/"

r = requests.get(url)
data = r.text
soup = BeautifulSoup(data, "html.parser")

table = soup.find('table')
rows = table.tbody.findAll('tr');

years = []
cpis = []

for row in rows:
  year = row.findAll('td')[0].get_text()
  if year.isdigit() and int(year) < 2017:
    years.append(int(year))
    cpis.append(float(row.findAll('td')[13].get_text()))

cpi_table = pd.DataFrame({
    "year": years,
    "avg_annual_cpi": cpis
})

cpi_table.head()

,year,avg_annual_cpi
0,1913,9.9
1,1914,10.0
2,1915,10.1
3,1916,10.9
4,1917,12.8


In [55]:
def get_real_value(nominalAmt, oldCpi, newCpi): # Функция перевода номинальных долларов в реальные
  realValue = (nominalAmt * newCpi) / oldCpi
  return realValue

CPI_2016 = float( cpi_table [ cpi_table ["year"] == 2016] ['avg_annual_cpi'])

float_columns = df.select_dtypes(include='float64').columns.tolist()
for column in float_columns:
    df[column] = df[column].astype('int') # Преобразовываем значения float в int

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4998 entries, 0 to 4997
Data columns (total 28 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   color                      4998 non-null   object
 1   director_name              4998 non-null   object
 2   num_critic_for_reviews     4998 non-null   int64 
 3   duration                   4998 non-null   int64 
 4   director_facebook_likes    4998 non-null   int64 
 5   actor_3_facebook_likes     4998 non-null   int64 
 6   actor_2_name               4998 non-null   object
 7   actor_1_facebook_likes     4998 non-null   int64 
 8   gross                      4998 non-null   int64 
 9   genres                     4998 non-null   object
 10  actor_1_name               4998 non-null   object
 11  movie_title                4998 non-null   object
 12  num_voted_users            4998 non-null   int64 
 13  cast_total_facebook_likes  4998 non-null   int64 
 14  actor_3_

Преобразуем значения столбцов ‘budget’ и ‘gross’ для каждого фильма в реальные долларовые эквиваленты 2016 г:

In [56]:
real_domestic_gross = []
real_budget_values = []

for index, row in df.iterrows():
  gross = row['gross']
  budget = row['budget']
  year = row['title_year']
  cpi = float(cpi_table[cpi_table['year'] == 2016]['avg_annual_cpi'])

  real_gross = get_real_value(gross, cpi, CPI_2016)
  real_budget = get_real_value(budget, cpi, CPI_2016)
  real_domestic_gross.append(real_gross)
  real_budget_values.append(real_budget)

df["real_domestic_gross"] = real_domestic_gross
df["real_budget"] = real_budget_values

Посчитаем окупаемость инвестиций (ROI) и абсолютную прибыль для каждого фильма. ROI покажет, какая киностудия получила наибольшую прибыль, исходя из первоначального бюджета фильма, и будет полезен при оценке успеха фильма в экономическом смысле.

In [57]:
profits = []
roi_vals = []

for index, row in df.iterrows():
  profit = row['real_domestic_gross'] - row['real_budget']
  budget = row['real_budget']
  num = profit - budget
  den = budget
  roi = (num/den) * 100

  profits.append(profit)
  roi_vals.append(roi)

df['profit'] = profits
df['roi'] = roi_vals

# **Проверка гипотез**

---



# **1. Правда, что популярность режиссёра в социальных сетях положительно влияет на рейтинг фильма?**

In [58]:
check_table = df [[
    'director_name',
    'movie_title',
    'director_facebook_likes',
    'imdb_score'
]]

check_table.head(10)

,director_name,movie_title,director_facebook_likes,imdb_score
0,James Cameron,Avatar,0,7
1,Gore Verbinski,Pirates of the Caribbean: At World's End,563,7
2,Sam Mendes,Spectre,0,6
3,Christopher Nolan,The Dark Knight Rises,22000,8
4,Doug Walker,Star Wars: Episode VII - The Force Awakens ...,131,7
5,Andrew Stanton,John Carter,475,6
6,Sam Raimi,Spider-Man 3,0,6
7,Nathan Greno,Tangled,15,7
8,Joss Whedon,Avengers: Age of Ultron,0,7
9,David Yates,Harry Potter and the Half-Blood Prince,282,7


Нельзя сказать, что чем популярнее режиссёр в социальных сетях, тем успешнее его фильм. Например Andrew Stanton и Sam Raimi популярнее чем Nathan Greno, однако его фильм оказался успешнее.

## **2. Правда, что чем больше бюджет фильма, тем выше его рейтинг?**

In [59]:
rate_table = df [[
    'movie_title',
    'budget',
    'imdb_score'
]]

rate_table.head(10)

,movie_title,budget,imdb_score
0,Avatar,237000000,7
1,Pirates of the Caribbean: At World's End,300000000,7
2,Spectre,245000000,6
3,The Dark Knight Rises,250000000,8
4,Star Wars: Episode VII - The Force Awakens ...,20000000,7
5,John Carter,263700000,6
6,Spider-Man 3,258000000,6
7,Tangled,260000000,7
8,Avengers: Age of Ultron,250000000,7
9,Harry Potter and the Half-Blood Prince,250000000,7


Нельзя сказать, что чем больше бюджет фильма, тем выше его рейтинг. Например Avatar  имеет меньший бюджет чем Spectre, однако его рейтинг оказался выше.

# **3. Правда, что фильмы с более популярными актёрами в соцсетях имеют более высокий рейтинг?**

In [62]:
df['total_actor_likes'] = df['actor_1_facebook_likes'] + df['actor_2_facebook_likes'] + df['actor_3_facebook_likes']

true_table = df [[
    'movie_title',
    'total_actor_likes',
    'imdb_score'
]]

true_table.head(10)

,movie_title,total_actor_likes,imdb_score
0,Avatar,2791,7
1,Pirates of the Caribbean: At World's End,46000,7
2,Spectre,11554,6
3,The Dark Knight Rises,73000,8
4,Star Wars: Episode VII - The Force Awakens ...,515,7
5,John Carter,1802,6
6,Spider-Man 3,39000,6
7,Tangled,1636,7
8,Avengers: Age of Ultron,66000,7
9,Harry Potter and the Half-Blood Prince,46000,7


Нельзя сказать, что фильмы с более популярными актёрами в социальных сетях имеют более высокий рейтинг. Например актёры из фильма Spectre популярнее чем Avatar, однако его рейтинг оказался ниже.

# **Заключение.**

---



Были проверены следующие три гипотезы:


1.   Правда, что популярность режиссёра в социальных сетях положительно влияет на рейтинг фильма?
2.   Правда, что чем больше бюджет фильма, тем выше его рейтинг?
3.   Правда, что фильмы с более популярными актёрами в соцсетях имеют более высокий рейтинг?

Проанализировав, мы можем прийти к выводу, что:
1. Нельзя сказать, что чем популярнее режиссёр в социальных сетях, тем успешнее его фильм.
*Гипотеза оказалась неверной*
2. Нельзя сказать, что чем больше бюджет фильма, тем выше его рейтинг.
*Гипотеза оказалась неверной*
3. Нельзя сказать, что фильмы с более популярными актёрами в соцсетях имеют более высокий рейтинг.
*Гипотеза оказалась неверной*




In [63]:
cpi_table.tail().to_csv("Lab№1 Tsiku FIT-221 cpi_table.csv", index=False)
df.to_csv("Lab№1 Tsiku FIT-221 movie_processed.csv", index=False)